Выводы:

1) Таблица market_df:
    - среднемесячная маркетинговая активность за последние 6 месяцев распределена нормально, в основном  4 раза в месяц. В редких случаях - 1-2 раза (за границами межквартильного размаха - выбросы). Значит всем клиентам (звонки, рассылки и пр) уделяется по 4 мероприятия в месяц.

    - маркетинговая активность в текущем месяце - строго 4

    - количество дней с момента регистрации пользователя тоже распределено нормально, половина всех пользователей зарегистрировалась от 400 до 800 дней назад (или от 1 года до 2,5 лет назад)

    - доля акционных покупок в месяц смещена вправо. Половина всех значений лежит от 19 до 25%, Нормально считается, если 50% товаров пользователь покупает не по акции (или по акции).  Однако встречаются выбросы - у некоторых покупателей доля покупок по акции составляет от 60 до 100%. Их нужно будет отсмотреть отдельно

    - средний просмотр категорий за визит пользователем - чаще всего 3. Основной набор - от 2 до 5

    - неоплаченные продукты за квартал - в основном от 1 до 5 товаров. Выбросы - более 8 товаров.

    - число сбоев, которые коснулись покупателя во время покупок - 50% пользователей испытывают от 3 до 6. 

    - среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца. - половина пользователей посещает от 5 до 11 страниц, Нормальным считается посещение от 2 до 20 страниц. Самое частое значение - 8 страниц за один визит на сайт

    - по активности пользователей за последние три месяца - у 40% активность снизилась

    - 30% клиентов из сектора премиум сервиса

    - 30% клиентов не дали согласия на получение рассылок

    - наиболее популярные товары у покупателей это Товары для детей и домашний текстиль. наименее популярные - мелкая быт техника и посуда

2) Таблица market_money_df:
    - прибыль за пред-предпоследний, предпоследний и последний месяцы распределена нормально
    - самая популярная выручка в месяц - в пределах от 4800 до 5200
    - есть несколько значений выбросов - менее 3000 и более 7000 в месяц

3) Таблица market_time_df_tr:
    - проведенное время на сайте за период прошлого и текущего месяца - лежит в границах от 5 до 23 минут
    - половина всех пользователей проводят от 10 до 17 минут, чаще всего - 13 минут

4) Таблица money_df:
    - среднемесячная прибыль с каждого покупателя лежит в пределах от 1,5 до 6,5 - остальное уже выбросы
    - половина покупателей приносит магазину прибыль от 3.3 до 4.7 , чаще всего  - 4 

In [ ]:
def plot_scatter_for_df(some_df,feat1, feat2, target_feature):

    # Создаем scatter-plot с линиями тренда для каждой группы точек
    fig = px.scatter(some_df, x=feat1, y=feat2, color= target_feature, #symbol=target_feature,
                    title= f'График рассеяния  {feat1} по {feat2} с учетом признака "{target_feature}"',
                    labels={feat1: feat1.capitalize(), feat2: feat2.capitalize()},
                    color_discrete_map={'Снизилась': 'blue', 'Прежний уровень': 'yellow'},  # Цвета точек для двух значений target_feat
    #                symbol_map={'Снизилась': 'circle', 'Прежний уровень': 'square'})  # Формы точек для двух значений target_feat
                    )


    # Фильтруем данные для каждой группы точек
    data_yellow = some_df[some_df[target_feature] == 'Прежний уровень']
    data_blue = some_df[some_df[target_feature] == 'Снизилась']

    # Добавляем линию тренда для красных точек
    fig.add_trace(px.scatter(data_yellow, 
                            x=feat1, 
                            y=feat2, 
                            trendline='ols',
                            trendline_color_override= 'orange',
                            color_continuous_scale=False).data[1])

    # Добавляем линию тренда для синих точек
    fig.add_trace(px.scatter(data_blue, 
                            x=feat1, 
                            y=feat2, 
                            trendline='ols',
                            trendline_color_override= 'blue',
                            color_continuous_scale=False).data[1])


    # Добавляем описание легенды

    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', marker=dict(color='orange'), 
                                showlegend=True, name=f'линия тренда для {target_feature} = Прежний уровень'))

    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', marker=dict(color='blue'), 
                                showlegend=True, name=f'линия тренда для {target_feature} = Снизилась'))

    # Перемещаем легенду над графиком
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))


    # Показываем график
    fig.show()


In [ ]:
plot_scatter_for_df(some_df= df_market,
                    feat1='last_last_rev',
                    feat2='current_dur_minutes', 
                    target_feature = 'consum_activity')

In [ ]:
target_feature = 'consum_activity'
list_of_columns = df_market.drop(['id', target_feature], axis=1)

list_of_err = []

for pair_of_cols in list_of_pairs:
    feat1= pair_of_cols[0]
    feat2 = pair_of_cols[1]

    if feat1 in  ['service_type', 'is_allow_report', 'pop_cat_goods'] or feat2 in  ['service_type', 'is_allow_report', 'pop_cat_goods']:
        continue
    try:
        plot_scatter_for_df(some_df= df_market,
                feat1=feat1,
                feat2=feat2, 
                target_feature = target_feature)

    except:
        list_of_err.append(pair_of_cols)
        continue



In [ ]:
from itertools import combinations

list_of_pairs = list(combinations(df_market.drop(['id', 'consum_activity'], axis=1).columns,
                                r=2))

list_of_pairs

In [ ]:
def plot_scatter_new(some_df, feat1, feat2, target_feature):

        # Разделяем данные на две группы в соответствии со значением target_feature
    good_data = some_df[some_df[target_feature] == 'Прежний уровень']
    bad_data = some_df[some_df[target_feature] == 'Снизилась']

    # Создаем scatter-plot
    plt.figure(figsize=(12, 4))

    # Создаем линии тренда для обеих групп точек
    def plot_trendline(data, color, label):
        x = data[feat1]
        y = data[feat2]
        A = np.vstack([x, np.ones(len(x))]).T
        m, c = np.linalg.lstsq(A, y, rcond=None)[0]
        plt.plot(x, m*x + c, color=color, linestyle='-', label=label)

    

    # Отображаем точки для обеих групп
    plt.scatter(bad_data[feat1], bad_data[feat2], color='tomato', label='Снизилась')
    plt.scatter(good_data[feat1], good_data[feat2], color='blueviolet', label="Прежний уровень")

    # Отображаем трендлинии для обеих групп
    plot_trendline(good_data, 'blue', 'Тренд-линия "Прежний уровень"')
    plot_trendline(bad_data, 'red', 'Трендлиния "Снизилась"')

    # Добавляем метки и заголовки
    plt.xlabel(f'{feat1}')
    plt.ylabel(f'{feat2}')
    plt.title(f'Scatter Plot для {feat1} по {feat2}')

    # Добавляем легенду
    plt.legend()

    # Показываем график
    plt.grid(True)
    plt.show()